# Parte 2
# Bactéria: Helicobacter pylori
Modelo: iIT341

Grupo 2: Diogo Cachetas, Joana Gabriel, João Monteiro, Juliana Pereira, Sara Boaventura, Teresa Coimbra

____________________________

a) É um organismo patogénico gástrico do ser Humano e está associado a patologias tais como: adoenocarcinoma gástrico, com a doença ulcerosa péptica e com a dispepsia funcional ou dispepsia não ulcerosa. Isto ocorre porque a bactéria diminui a capacidade de reparação da mucosa, tornando-a mais susceptível ao ácido gástrico

In [1]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator

In [2]:
model = read_sbml_model('./iIT341.xml')

In [3]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ala__D_e,EX_ala__D_e,10,3,38.15%
ala__L_e,EX_ala__L_e,10,3,38.15%
arg__L_e,EX_arg__L_e,2.151,6,16.41%
h_e,EX_h_e,3.981,0,0.00%
his__L_e,EX_his__L_e,0.06235,6,0.48%
ile__L_e,EX_ile__L_e,0.1912,6,1.46%
leu__L_e,EX_leu__L_e,0.2965,6,2.26%
met__L_e,EX_met__L_e,0.2054,5,1.31%
o2_e,EX_o2_e,12,0,0.00%
pheme_e,EX_pheme_e,4.157E-06,34,0.00%


# b) Taxa de crescimento específico nas condições adequadas de crescimento do organismo e quais os produtos excretados.

Condições ambientais de crescimento de *H.pylori*: considerada microaerófila, o seu crescimento dá-se entre 5 a 19% de oxigénio e 5-10% de CO2, sendo que também consegue crescer em ambientes aeróbios (2).

In [4]:
test = model.copy()
test.reactions.get_by_id("EX_o2_e").lower_bound = -12   #Condições base de O2 do modelo
test.reactions.get_by_id("EX_ala__L_e").lower_bound = -10
test.reactions.get_by_id("EX_ala__D_e").lower_bound = -10

In [5]:
print(test.summary())

Objective
1.0 BIOMASS_HP_published = 0.6928126934734461

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
  ala__D_e  EX_ala__D_e        10         3  38.15%
  ala__L_e  EX_ala__L_e        10         3  38.15%
  arg__L_e  EX_arg__L_e     2.151         6  16.41%
       h_e       EX_h_e     3.981         0   0.00%
  his__L_e  EX_his__L_e   0.06235         6   0.48%
  ile__L_e  EX_ile__L_e    0.1912         6   1.46%
  leu__L_e  EX_leu__L_e    0.2965         6   2.26%
  met__L_e  EX_met__L_e    0.2054         5   1.31%
      o2_e      EX_o2_e        12         0   0.00%
   pheme_e   EX_pheme_e 4.157E-06        34   0.00%
      pi_e      EX_pi_e     0.642         0   0.00%
    pime_e    EX_pime_e 4.157E-06         7   0.00%
     thm_e     EX_thm_e 4.157E-06        12   0.00%
  val__L_e  EX_val__L_e    0.2786         5   1.77%

Secretion
---------
Metabolite     Reaction       Flux  C-Number  C-Flux
    amob_c    DM_amob_c -4.157E-06        15   0.00%
  hmfurn_c  DM_hmfurn_

# Os produtos excretados: 
- succinato (17.52%), 
- acetato (44.14%),
- CO2 (25.14%), 
- ornitina (11.34%)
- SAH -o precursor da homocisteina- (1.23%).
- A bactéria produz e secreta **urease**. A urease é hidrolisada em ureia, CO2 e NH4, permindo-lhe neutralizar o ácido gástrico localmente e, desta forma, colonizar o epitélio gástrico(3). 
- SK_ahcys_c (SAH) - é o precursor da homocisteina --> Um nível alto de homocisteina no sangue leva a inflamações dos vasos sanguinios e danos nas células endoteliais

________________________________________________________________________________



In [5]:
simul = get_simulator(model)
result = simul.simulate(method='FBA')

In [6]:
simul.objective

{'BIOMASS_HP_published': 1.0}

In [8]:
test2 = model.copy()
flu3 = []
print('Single Gene Knockout')
print('')
for i in range(len(test2.genes)):
    #Distinguir entre genes essenciais e não essenciais
    if test2.genes[i].id in simul.essential_genes:
        print(f'Gene essencial: {test2.genes[i].name}')
        test2.genes[i].knock_out()
        gene = [test2.genes[i]]
        solution = test2.optimize()
        solution.objective_value
        print('fluxo biomassa',solution.fluxes['BIOMASS_HP_published'])
        gene.append(solution.fluxes['BIOMASS_HP_published'])
        flu3.append(gene)
        test2 = model.copy()
        print('')
    else:
        test2.genes[i].knock_out()
        print(test2.genes[i])
        gene = [test2.genes[i]]
        solution = test2.optimize()
        solution.objective_value
        print('fluxo biomassa',solution.fluxes['BIOMASS_HP_published'])
        gene.append(solution.fluxes['BIOMASS_HP_published'])
        flu3.append(gene)
        test2 = model.copy()
        print('')

Single Gene Knockout

HP0056
fluxo biomassa 0.6831117287633226

HP0690
fluxo biomassa 0.6928126934659237

Gene essencial: HP0370
fluxo biomassa 0.0

Gene essencial: HP0950
fluxo biomassa 0.0

Gene essencial: HP0371
fluxo biomassa 0.0

Gene essencial: HP0557
fluxo biomassa 0.0

HP0903
fluxo biomassa 0.5421330938206107

Gene essencial: HP0202
fluxo biomassa 0.0

HP0779
fluxo biomassa 0.6928126934659237

HP0930
fluxo biomassa 0.6928126934659237

HP0808
fluxo biomassa 0.6928126934659237

HP1045
fluxo biomassa 0.6928126934659237

Gene essencial: HP0587
fluxo biomassa 0.0

Gene essencial: adk
fluxo biomassa 0.0

HP1180
fluxo biomassa 0.6928126934659237

HP0572
fluxo biomassa 0.6916221254771253

Gene essencial: HP1112
fluxo biomassa 0.0

Gene essencial: HP0255
fluxo biomassa 0.0

Gene essencial: HP0859
fluxo biomassa 0.0

Gene essencial: HP0089
fluxo biomassa 0.0

Gene essencial: HP0106
fluxo biomassa 0.0

HP1091
fluxo biomassa 0.6928126934659237

Gene essencial: ddl
fluxo biomassa 0.0

HP139

In [9]:
cobra_model2 = test.copy()
flu = []
essenc = []
vias_rip = {}
# Função que separa reações importantes de H.pyloris e separa as Essenciais das não essenciais
for i in range(len(cobra_model2.reactions)):  
    if cobra_model2.reactions[i].id in simul.essential_reactions:  #Reações essenciais
        print(i)
        print(f'a reação {cobra_model2.reactions[i].id} é essencial!')
        cobra_model2.reactions[i].lower_bound = 0 # Dar o Knockout das vias consistem em limitar os bounds delas
        cobra_model2.reactions[i].upper_bound = 0 #  deste modo conseguimos saber, das reações não esseciais as que influênciam o fluxo de biomassa
        print(cobra_model2.reactions[i])
        gene = [cobra_model2.reactions[i]]
        solution = cobra_model2.optimize()
        solution.objective_value
        gene.append(solution.objective_value)
        print('fluxo biomassa',solution.fluxes['BIOMASS_HP_published'])
        gene.append(solution.fluxes['BIOMASS_HP_published'])
        if solution.fluxes['BIOMASS_HP_published'] < 0.20:
            vias_rip[cobra_model2.reactions[i].id]= solution.objective_value
        flu.append(gene)
        cobra_model2 = test.copy()
        print('')
    else:
        #reações não essenciais
        print(i)
        cobra_model2.reactions[i].lower_bound = 0 # Dar o Knockout das vias consistem em limitar os bounds delas
        cobra_model2.reactions[i].upper_bound = 0 #  deste modo conseguimos saber, das reações não esseciais as que influênciam o fluxo de biomassa
        print(cobra_model2.reactions[i])
        gene = [cobra_model2.reactions[i]]
        solution = cobra_model2.optimize()
        solution.objective_value
        gene.append(solution.objective_value)
        print('fluxo biomassa',solution.fluxes['BIOMASS_HP_published'])
        gene.append(solution.fluxes['BIOMASS_HP_published'])
        if solution.fluxes['BIOMASS_HP_published'] < 0.20:
            vias_rip[cobra_model2.reactions[i].id]= solution.objective_value
        flu.append(gene)
        cobra_model2 = test.copy()
        print('')

0
CPPPGO: cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c
fluxo biomassa 0.6928126934734461

1
CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
fluxo biomassa 0.6928126934734461

2
a reação CTPS1 é essencial!
CTPS1: atp_c + nh4_c + utp_c --> adp_c + ctp_c + 2.0 h_c + pi_c
fluxo biomassa 0.0

3
a reação CYOO_HP é essencial!
CYOO_HP: 4.0 focytcc553_c + 5.98 h_c + o2_c --> 4.0 ficytcc553_c + 1.99 h2o_c + 2.0 h_e + 0.005 o2s_c
fluxo biomassa 0.0

4
a reação CYSS é essencial!
CYSS: acser_c + h2s_c --> ac_c + cys__L_c + h_c
fluxo biomassa 0.0

5
CYSabc: atp_c + cys__L_e + h2o_c --> adp_c + cys__L_c + h_c + pi_c
fluxo biomassa 0.6928126934734461

6
CYTDt2: cytd_e + h_e --> cytd_c + h_c
fluxo biomassa 0.6928126934734461

7
a reação CYTK1 é essencial!
CYTK1: atp_c + cmp_c --> adp_c + cdp_c
fluxo biomassa 0.0

8
a reação ADMDC é essencial!
ADMDC: amet_c + h_c --> ametam_c + co2_c
fluxo biomassa 0.0

9
DAAD: ala__D_c + fad_c + h2o_c --> fadh2_c + nh4_c + pyr_c
fluxo biomassa 0.69

In [12]:
simul.essential_reactions

['CTPS1',
 'CYOO_HP',
 'CYSS',
 'CYTK1',
 'ADMDC',
 'EX_pime_e',
 'ADSL1r',
 'ADSL2r',
 'ADSS',
 'DAPDC',
 'DAPE',
 'EX_pi_e',
 'DASYN_HP',
 'DB4PS',
 'AGMHE',
 'DBTS',
 'AGMT',
 'AHCYSNS',
 'A5PISO',
 'AICART',
 'AIRC2',
 'AIRC3',
 'ALAALAr',
 'ACCOAC',
 'AMAOTr',
 'DDPA',
 'DHDPRy',
 'DHDPS',
 'DHFR',
 'ADCL',
 'DHFS',
 'AMMQT6',
 'ANPRT',
 'ADK1',
 'DHNAOT2',
 'DHNPA',
 'AOXSr',
 'APRAUR',
 'DHORD3',
 'ARGDC',
 'ARGN',
 'DHORTS',
 'ARGt2r',
 'DHPPDA',
 'ASAD',
 'ASNS2',
 'DHPTDCs',
 'DHQTi',
 'ASP1DC',
 'ASPCT',
 'ASPK',
 'EX_his__L_e',
 'EX_ile__L_e',
 'ASPO2',
 'ASPTA',
 'DHQS',
 'EX_leu__L_e',
 'DKMPPD',
 'BC10_new',
 'EX_met__L_e',
 'DMATT',
 'BTS2',
 'BIOMASS_HP_published',
 'C140SN',
 'C160SN',
 'DMPPS',
 'C180SN',
 'C181SN',
 'C190cSN',
 'EX_o2_e',
 'DM_hmfurn_c',
 'CBPS',
 'CDPMEK',
 'DNMPPA',
 'CHCOAL',
 'CHORM',
 'EX_thm_e',
 'CHORS',
 'DNTPPA',
 'CLPNS_HP',
 'DPCOAK',
 'DPR',
 'EX_val__L_e',
 'DTMPK',
 'FBA',
 'GCALDD',
 'FBP',
 'GF6PTA',
 'DXPRIi',
 'GFUCS',
 'GHMT2r',
 

In [9]:
len(simul.essential_reactions)

117

In [15]:
model.metabolites.nh4_e

Metabolite identifier,nh4_e
Name,Ammonium
Memory address,0x01c183657188
Formula,H4N
Compartment,e
In 2 reaction(s),"NH4t, EX_nh4_e"


In [16]:
model.reactions.NH4t

Reaction identifier,NH4t
Name,Ammonia reversible transport
Memory address,0x01c183b0a9c8
Stoichiometry,nh4_e <=> nh4_c Ammonium <=> Ammonium
GPR,
Lower bound,-999999.0
Upper bound,999999.0


In [19]:
model.reactions.UREA.summary()

____________________________________